In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
import seaborn as sns
import random
from keras.models import Model 
from keras.layers import Dense, Embedding, Input, Lambda, BatchNormalization, Dropout, Add, Concatenate
from keras.layers.noise import GaussianNoise
from keras.callbacks import EarlyStopping, Callback
from keras import regularizers
from keras.optimizers import adam
import keras.backend as K
import tensorflow as tf
import math
from tqdm import tqdm
from tensorflow.compat.v1 import set_random_seed 
import gc

from keras.utils import plot_model
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull

class GoldenTimer:
    def __init__(self, show=True):
        self.start_time = time.time()
        self.show = show

    def time(self, print_str):
        duration = time.time() - self.start_time
        if self.show:
            print(print_str, duration)
        self.start_time = time.time()

K.clear_session()
random.seed(2019)
set_random_seed(2019)

/kaggle/input/nfl-big-data-bowl-2020/train.csv
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/sample_submission.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/test.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/__init__.py


Using TensorFlow backend.


In [2]:
#### Feature Engineering ####
#train_df = pd.read_csv("train.csv")
train_df = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv")

exclude_global_feature = ['Week','Humidity', 'Stadium', 'Location', 'Turf', 'GameWeather',
                          'Temperature',  'WindSpeed', 'WindDirection', 'Season',
                           'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'YardLine_std','YardLine']#
exclude_personal_feature = [ 'NflId',"player's name",'NflIdRusher', 'Orientation', 'Dir', 'PlayerWeight', 'PlayerHeight']
exclude_feature = ['Quarter','Yards', 'PlayId', 'GameId'] + exclude_global_feature + exclude_personal_feature

category_features = ['DefendersInTheBox', 'Position', 'OffenseFormation', 'DefensePersonnel']# 'Quarter', 'Down',,
#player_continuous_feature = ['speed_sin_Dir', 'speed_cos_Dir', 'acc_sin_Dir', 'acc_cos_Dir', 'X', 'Y', 'dx', 'dy']
player_continuous_feature = ['speed_sin_Dir', 'speed_cos_Dir', 'acc_sin_Dir', 'acc_cos_Dir', 'X', 'Y', "x1", "y1", "x2", "y2"]
absolute_player_continuous_feature = []

# TODO: age
player_category_feature = ['Position','JerseyNumber']
# TODO: JerseyNumber

# team features 
friend_continuous_features = ['friend' + str(i) + '_cont_' + cont_feature for cont_feature in player_continuous_feature for i in range(10)]
friend_category_features = ['friend' + str(i) + '_cat#' + cont_feature for cont_feature in player_category_feature for i in range(10)]
ravel_continuous_features = ['ravel' + str(i) + '_cont_' + cont_feature for cont_feature in player_continuous_feature for i in range(11)]
ravel_category_features = ['ravel' + str(i) + '_cat#' + cont_feature for cont_feature in player_category_feature for i in range(11)]
friends_position = ["friend" + str(i) + "_position" for i in range(10)]
ravel_position = ["ravel" + str(i) + "_position" for i in range(11)]

team_player_feature = friend_continuous_features + friend_category_features + ravel_continuous_features + ravel_category_features


class GroupDataWrap(object):
    def __init__(self, np_arr, columns, column_map=None):
        self.np_arr = np_arr
        self.columns = columns
        self.column_map = column_map if column_map is not None else dict([(pair[1], pair[0]) for pair in enumerate(columns)])
        
    def __getitem__(self, col):
        if isinstance(col, str):
            return self.np_arr[:,self.column_map[col]]
        else:
            return GroupDataWrap(self.np_arr[col,:], self.columns)

def fe(df):
    mean_old, mean_new, std_old, std_new = 2.435519556913685, 2.7570316419451517, 1.2929623410155855, 1.4551321358655551
    df["S"] = np.where(df["Season"] >= 2018, (df["S"]-mean_new) / std_new * std_old + mean_old, df["S"])
    mean_old, mean_new, std_old, std_new = 1.5895792207792045, 1.7819953460610594, 0.8795106467756848, 1.060305722313926
    df["A"] = np.where(df["Season"] >= 2018, (df["A"]-mean_new) / std_new * std_old + mean_old, df["A"])
    
    df["VisitorTeamAbbr"] = np.where(df["VisitorTeamAbbr"] == "ARI", "ARZ", df["VisitorTeamAbbr"])
    df["HomeTeamAbbr"] = np.where(df["HomeTeamAbbr"] == "ARI", "ARZ", df["HomeTeamAbbr"])
    df["VisitorTeamAbbr"] = np.where(df["VisitorTeamAbbr"] == "BAL", "BLT", df["VisitorTeamAbbr"])
    df["HomeTeamAbbr"] = np.where(df["HomeTeamAbbr"] == "BAL", "BLT", df["HomeTeamAbbr"])
    df["VisitorTeamAbbr"] = np.where(df["VisitorTeamAbbr"] == "CLE", "CLV", df["VisitorTeamAbbr"])
    df["HomeTeamAbbr"] = np.where(df["HomeTeamAbbr"] == "CLE", "CLV", df["HomeTeamAbbr"])
    df["VisitorTeamAbbr"] = np.where(df["VisitorTeamAbbr"] == "HOU", "HST", df["VisitorTeamAbbr"])
    df["HomeTeamAbbr"] = np.where(df["HomeTeamAbbr"] == "HOU", "HST", df["HomeTeamAbbr"])

    df["PossessionTeam"] = np.where(df["PossessionTeam"] == "ARI", "ARZ", df["PossessionTeam"])
    df["FieldPosition"] = np.where(df["FieldPosition"] == "ARI", "ARZ", df["FieldPosition"])
    df["PossessionTeam"] = np.where(df["PossessionTeam"] == "BAL", "BLT", df["PossessionTeam"])
    df["FieldPosition"] = np.where(df["FieldPosition"] == "BAL", "BLT", df["FieldPosition"])
    df["PossessionTeam"] = np.where(df["PossessionTeam"] == "CLE", "CLV", df["PossessionTeam"])
    df["FieldPosition"] = np.where(df["FieldPosition"] == "CLE", "CLV", df["FieldPosition"])
    df["PossessionTeam"] = np.where(df["PossessionTeam"] == "HOU", "HST", df["PossessionTeam"])
    df["FieldPosition"] = np.where(df["FieldPosition"] == "HOU", "HST", df["FieldPosition"])
    
    df["team_abbr"] = np.where(df["Team"] == "home", df["HomeTeamAbbr"], df["VisitorTeamAbbr"])
    df["reval_abbr"] = np.where(df["Team"] == "away", df["HomeTeamAbbr"], df["VisitorTeamAbbr"])

    df.loc[df['PlayDirection'] == 'left', 'X'] = 120 - df.loc[df['PlayDirection'] == 'left', 'X']
    df.loc[df['PlayDirection'] == 'left', 'Y'] = (160 / 3) - df.loc[df['PlayDirection'] == 'left', 'Y']
    df.loc[df['PlayDirection'] == 'left', 'Orientation'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation'], 360)
    df.loc[df['PlayDirection'] == 'left', 'Dir'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir'], 360)    
        
    df['FieldPosition'].fillna('', inplace=True)
    
    df['speed_sin_Dir'] = df['S'] * np.sin(df['Dir'] / 180 * np.pi)
    df['speed_cos_Dir'] = df['S'] * np.cos(df['Dir'] / 180 * np.pi)
    
    df['acc_sin_Dir'] = df['A'] * np.sin(df['Dir'] / 180 * np.pi)
    df['acc_cos_Dir'] = df['A'] * np.cos(df['Dir'] / 180 * np.pi)
    
    df["dxdy_dir"] = (540 - (df["Dir"] + 90)) % 360
    df["dy"] = np.sin(np.deg2rad(df["dxdy_dir"]))
    df["dx"] = np.cos(np.deg2rad(df["dxdy_dir"]))
    df["x1"] = df["X"] + (df["dx"] * (df["S"]))
    df["y1"] = df["Y"] + (df["dy"] * (df["S"]))
    df["x2"] = df["X"] + (df["dx"] * (df["S"]*2))
    df["y2"] = df["Y"] + (df["dy"] * (df["S"]*2))
    df.drop(['dxdy_dir'], inplace=True, axis=1)
    
    df["is_old"] = np.where(df["Season"]<2018, 1, 0)
    
    df["YardLine_std"] = np.where(df["PossessionTeam"] == df["FieldPosition"], df["YardLine"]+10, 110-df["YardLine"])
    df['YardLine_std_diff'] = df['X'] - df['YardLine_std']
    
    grass_labels = tuple(['grass', 'natural grass', 'natural', 'naturall grass'])
    df['Grass'] = np.where(df.Turf.str.lower().isin([grass_labels]), 1, 0)


    rusher_df = df[df['NflId'] == df['NflIdRusher']]   
    
    # team features
    # fields: [rusher nfl id, PlayId] [friends fields] [reval fields]
    val_rows = []
    val_schema = ['NflId', 'PlayId'] + team_player_feature
    
    for groupname, group_df in (df.groupby(['PlayId'])):
        row_fields = []
        
        group_df = GroupDataWrap(group_df.to_numpy(), group_df.columns)
        
        runner = group_df[group_df['NflId'] == group_df['NflIdRusher']]   
        friends = group_df[(group_df['PossessionTeam'] == group_df['team_abbr']) & (group_df['NflId'] != group_df['NflIdRusher'])]
        ravel = group_df[group_df['PossessionTeam'] == group_df['reval_abbr']]
        
        row_fields.append(runner['NflId'][0])
        row_fields.append(runner['PlayId'][0])
        for cont_feature in player_continuous_feature:
            if cont_feature in absolute_player_continuous_feature:
                row_fields += friends[cont_feature].tolist()
            else:
                row_fields += (friends[cont_feature] - runner[cont_feature][0]).tolist()
        for cat_feature in player_category_feature:
            row_fields += friends[cat_feature].tolist()
        for cont_feature in player_continuous_feature:
            if cont_feature in absolute_player_continuous_feature:
                row_fields += ravel[cont_feature].tolist()
            else:
                row_fields += (ravel[cont_feature] - runner[cont_feature][0]).tolist()
        for cat_feature in player_category_feature:
            row_fields += ravel[cat_feature].tolist()
            
        val_rows.append(row_fields)
            
    team_feature_df = pd.DataFrame(val_rows, columns=val_schema)
    
    df = rusher_df.merge(team_feature_df, on=['NflId', 'PlayId'])

    return df


timer = GoldenTimer()
train_df.fillna(0, inplace=True)
train_df = fe(train_df)
print(train_df.shape)
timer.time("done fe")


Y_MAX = 54
X_MAX = 120
X_FORWARD = 20

RESOLUTION = 2

def DropoutDense(units, activation="linear", name=None, dropout_ratio=None):
    if dropout_ratio is None:
        return Dense(units, name=name, activation=activation)
    else:
        dense = Dense(units, activation=activation)
        dropout = Dropout(dropout_ratio, name=name)
        return lambda x: dropout(dense(x))


class NLF_NN(object):
    def __init__(self, continuous_features, category_features, label, epoch, batch_size=2000, 
                 common_embedding_size=8, specified_embedding_map=None, learning_rate=0.005, 
                 player_continuous_feature=None, player_category_feature=None):
        self.continuous_features = [f for f in continuous_features if f != label]
        self.category_features = category_features
        self.common_embedding_size = common_embedding_size
        self.specified_embedding_map = specified_embedding_map if specified_embedding_map is not None else dict()
        self.learning_rate = learning_rate
        self.label = label
        self.epoch = epoch
        self.batch_size = batch_size
        self.player_continuous_feature = player_continuous_feature
        self.player_category_feature = player_category_feature
        
        
    def crps_loss_by_label_tensor(self, y_true, y_pred):
        y_true = K.clip(K.cumsum(y_true, axis=-1), 0, 1)
        y_pred = K.clip(K.cumsum(y_pred, axis=-1), 0, 1)
        return K.mean(K.square(y_true - y_pred))
    
    def crps_loss_by_label_np(self, y_true, y_pred):
        return np.mean(np.square(y_true - y_pred))
    
    def make_category_embedding(self, cat_input, category_list, cat_size_map, name=None, embedding_size=None):
        def _inner_make_category_embedding(cat_input):
            cat_emb_res = []
            for i in range(len(category_list)):
                cat_field_name = category_list[i]
                emb_size = embedding_size if embedding_size is not None else self.specified_embedding_map.get(cat_field_name, self.common_embedding_size)
                cat_size = cat_size_map[cat_field_name]
                emb_tensor = Embedding(cat_size, emb_size)(cat_input[:,i])
                cat_emb_res.append(emb_tensor)
                
            return K.concatenate(cat_emb_res, axis=-1)
        
        return Lambda(_inner_make_category_embedding, name=name)(cat_input)
    
    def make_team_embedding(self, cont_features, cat_features, cat_size_map):
        # l1:256 cont dense
        # l2:combined cont and cat emb
        # l3:128 dense
        
        cont_dense = Dense(256, activation='relu')
        comb_dense_ = Dense(128, activation='relu')
        comb_bn = BatchNormalization()
        comb_dense = lambda x: comb_bn(comb_dense_(x))
        out_dense = DropoutDense(64, activation='relu')
        
        team_embeddings = []
        for i in range(len(cont_features)):
            hidden1 = cont_dense(cont_features[i])
#            cat_emb = self.make_category_embedding(cat_features[i], self.player_category_feature, cat_size_map, embedding_size=8)
#            combined_feature = Lambda(lambda xs: K.concatenate(xs, axis=-1))([hidden1, cat_emb])
#            hidden2 = comb_dense(combined_feature)
            hidden2 = comb_dense(hidden1)
            hidden3 = out_dense(hidden2)
            team_embeddings.append(Lambda(lambda x: K.expand_dims(x, 1))(hidden3))
                
        return team_embeddings
    
    def make_mix_team_embedding(self, cont_friend, cat_friend, cont_ravel, cat_ravel, cat_size_map):        
        friend_embeddings = self.make_team_embedding(cont_friend, cat_friend, cat_size_map)
        ravel_embeddings = self.make_team_embedding(cont_ravel, cat_ravel, cat_size_map)
        
        friend_weight_dense = Dense(1, activation='sigmoid')
        ravel_weight_dense = Dense(1, activation='sigmoid')
        
        
        friends_weights = []
        ravels_weights = []
        
        friend_mean_embedding = Lambda(lambda xs: K.mean(xs, axis=1))(Concatenate(axis=1)(friend_embeddings))
        ravel_mean_embedding = Lambda(lambda xs: K.mean(xs, axis=1))(Concatenate(axis=1)(ravel_embeddings))
        mean_embedding = Add()([friend_mean_embedding, ravel_mean_embedding])
                
        for palyer_emb in friend_embeddings:
            ca = Lambda(lambda xs: K.concatenate([xs[0], K.squeeze(xs[1], axis=1)]))([mean_embedding, palyer_emb])
            friends_weights.append(friend_weight_dense(ca))
            
        for palyer_emb in ravel_embeddings:
            ca = Lambda(lambda xs: K.concatenate([xs[0], K.squeeze(xs[1], axis=1)]))([mean_embedding, palyer_emb])
            ravels_weights.append(ravel_weight_dense(ca))
            
        friend_concated_weights = Lambda(lambda x: K.exp(x))(Concatenate(axis=-1)(friends_weights))
        friend_concated_weights = Lambda(lambda x: x / K.sum(x, axis=-1, keepdims=True))(friend_concated_weights)
        
        ravel_concated_weights = Lambda(lambda x: K.exp(x))(Concatenate(axis=-1)(ravels_weights))
        ravel_concated_weights = Lambda(lambda x: x / K.sum(x, axis=-1, keepdims=True))(ravel_concated_weights)
        
        friend_concated_team_embeddings = Concatenate(axis=1)(friend_embeddings)
        ravel_concated_team_embeddings = Concatenate(axis=1)(ravel_embeddings)
        
        friend_res = Lambda(lambda ps: K.mean(tf.matmul(K.expand_dims(ps[0], axis=1), ps[1]), axis=1))([friend_concated_weights, friend_concated_team_embeddings])
        ravel_res = Lambda(lambda ps: K.mean(tf.matmul(K.expand_dims(ps[0], axis=1), ps[1]), axis=1))([ravel_concated_weights, ravel_concated_team_embeddings])
        
        return friend_res, ravel_res
        
        
    def build_muti_class_model(self, cat_size_map):
        # build nn structure of discrete feature
        cat_embedding = self.make_category_embedding(self.category_input, self.category_features, cat_size_map, name="multiclass_cat_embedding")
        
        friend_team_embedding, ravel_team_embedding = self.make_mix_team_embedding(self.continuous_friend_input, self.category_friend_input, self.continuous_ravel_input, self.category_ravel_input, cat_size_map)
        
        hidden_layer1 = Dense(256, name="sigmoid_hidden1", activation='relu')(self.continuous_input)
        combined_feature = Lambda(lambda xs: K.concatenate(xs, axis=-1), name="combine_cont_dis")([hidden_layer1, friend_team_embedding, ravel_team_embedding, cat_embedding])
        hidden_layer2 = DropoutDense(256, name="sigmoid_hidden2", activation='relu')(combined_feature)

        # final output
        hidden_layer4 = DropoutDense(128, name="sigmoid_hidden4", dropout_ratio=0.5, activation='relu')(hidden_layer2)
        final_output = Dense(199, name="output_dense1", activation='softmax')(hidden_layer4)     
        final_output = Lambda(lambda x: K.clip(x, 1e-8, 1), name="output_dense1_clip")(final_output)

        return final_output
    
    
    def build_model(self, cat_size_map):
        self.continuous_input = Input((len(self.continuous_features), ), dtype=tf.float32, name="continuous_input")
        self.category_input = Input((len(self.category_features), ), dtype=tf.int32, name="category_input")
        
        # team friend features
        self.continuous_friend_input = [Input((len(self.player_continuous_feature), ),  dtype=tf.float32, name="friend_continuous_input" + str(i)) for i in range(10)]
        self.category_friend_input = [Input((len(self.player_category_feature), ),  dtype=tf.int32, name="friend_category_input" + str(i)) for i in range(10)]

        # team ravel features
        self.continuous_ravel_input = [Input((len(self.player_continuous_feature), ),  dtype=tf.float32, name="ravel_continuous_input" + str(i)) for i in range(11)]
        self.category_ravel_input = [Input((len(self.player_category_feature), ),  dtype=tf.int32, name="ravel_category_input" + str(i)) for i in range(11)]
  
        # clip value avoid overflow
        self.output = self.build_muti_class_model(cat_size_map)
                
        self.model = Model(inputs=[self.continuous_input, self.category_input] + 
                           self.continuous_friend_input + self.category_friend_input +
                           self.continuous_ravel_input + self.category_ravel_input,
                           outputs=self.output)
    
        optimizer = adam(lr=self.learning_rate, clipvalue=0.005)
        
        # compile model
        self.model.compile(optimizer, loss="categorical_crossentropy", metrics=[self.crps_loss_by_label_tensor])
        
    def make_y_vec(self, y_list):
        y_list =np.round(y_list).astype('int32') + 99
        target = np.zeros((len(y_list), 199))
        target[range(len(y_list)),y_list] = 1
        
        return target
        
    def train_model(self, train_df, validate_df=None):
        # print(self.continuous_features)
        x_continuous = train_df[self.continuous_features]
        x_category = train_df[self.category_features]
        x_friend_continuours = [train_df[[f for f in friend_continuous_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        x_friend_category = [train_df[[f for f in friend_category_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        x_ravel_continuours = [train_df[[f for f in ravel_continuous_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        x_ravel_category = [train_df[[f for f in ravel_category_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        # print(x_friend_continuours[0].columns)
        # print(x_ravel_continuours[0].shape)
        
        validate_x_continuous = validate_df[self.continuous_features]
        validate_x_category = validate_df[self.category_features]
        validate_x_friend_continuours = [validate_df[[f for f in friend_continuous_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        validate_x_friend_category = [validate_df[[f for f in friend_category_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        validate_x_ravel_continuours =  [validate_df[[f for f in ravel_continuous_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        validate_x_ravel_category = [validate_df[[f for f in ravel_category_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        
        y_vector = self.make_y_vec(train_df[self.label])
        validate_y_vector = self.make_y_vec(validate_df[self.label])
        
        xs = [x_continuous, x_category] + x_friend_continuours + x_friend_category + \
            x_ravel_continuours + x_ravel_category
        
        validate_xs = [validate_x_continuous, validate_x_category] + validate_x_friend_continuours + \
                        validate_x_friend_category  + \
                        validate_x_ravel_continuours + validate_x_ravel_category
                
        es = EarlyStopping(monitor='val_crps_loss_by_label_tensor', 
                       mode='min',
                       verbose=2, 
                       restore_best_weights=True,
                       patience=10)

        self.model.fit(xs, y_vector, epochs=self.epoch, batch_size=self.batch_size, 
                       callbacks=[es], validation_data=(validate_xs, validate_y_vector))
        
        
    def score(self, validate_df):
        y = self.predict(validate_df)
        y_vector = np.cumsum(self.make_y_vec(validate_df[self.label]), axis=-1)
        
        return y, y_vector, self.crps_loss_by_label_np(y, y_vector)
       
        
    def predict(self, test_df):
        x_continuous = test_df[self.continuous_features]
        x_category = test_df[self.category_features]
        x_friend_continuours = [test_df[[f for f in friend_continuous_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        x_friend_category = [test_df[[f for f in friend_category_features if f.startswith("friend"+str(i)+"_")]]  for i in range(10)]
        x_ravel_continuours = [test_df[[f for f in ravel_continuous_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        x_ravel_category = [test_df[[f for f in ravel_category_features if f.startswith("ravel"+str(i)+"_")]]  for i in range(11)]
        
        xs = [x_continuous, x_category] + x_friend_continuours + x_friend_category + \
              x_ravel_continuours + x_ravel_category
        
        y = self.model.predict(xs)
        y = np.clip(np.cumsum(y, axis=-1), 0, 1)
        
        # modify by YardLine_std
        max_to_go = np.array(110 - test_df['YardLine_std'] + 0.5).astype(np.int32)
        target_index = np.array([np.arange(199)] * len(max_to_go))
        # print(max_to_go)
        y[target_index-99 >= np.array([max_to_go]).T] = 1
        y[target_index-99 <= np.array([max_to_go]).T - 110] = 0
        
        return y

class KFoldByKey(object):
    def __init__(self, nfold, shuffle=False):
        self.nfold = nfold
        self.shuffle= shuffle
        
    def __flatten(self, list2d):
        ret = []
        for i in list2d:
            ret += i
        return ret
        
    def split(self, df_col):        
        key_values = list(set(df_col))
        split_step = float(len(key_values)) / self.nfold
        range_point = [int(i * split_step) for i in range(self.nfold)] + [len(key_values)]
        key_splits = [key_values[range_point[i]:range_point[i+1]] for i in range(self.nfold)]
        
        for i in range(self.nfold):
            test_keys = key_splits[i]
            train_keys = self.__flatten([key_splits[j] for j in range(self.nfold) if j != i])
            
            yield (np.where(df_col.isin(train_keys)), np.where(df_col.isin(test_keys)))


class CategoryFeatureEncoder(object):
    def __init__(self, category_features):
        self.label_encoders = dict()
        self.category_features = category_features
        self.cat_size = dict()

    def fit(self, df):
        for column in self.category_features:
            suffix = column.split('#')[-1]
            if suffix not in self.label_encoders:
                related_cols = [col for col in self.category_features if col.split("#")[-1] == suffix]
                related_category = []
                for col in related_cols:
                    related_category += list(set(df[col]))
                related_category = list(set(related_category))
                _, indexer = pd.DataFrame({"x": related_category})['x'].factorize()
                self.cat_size[suffix] = indexer.size + 2
                self.label_encoders[suffix] = indexer

    def transform(self, df):
        df = df.copy()
        for col in self.category_features:
            suffix = col.split('#')[-1]
            indexer = self.label_encoders[suffix]
            df[col] = indexer.get_indexer(df[col]) + 2

        return df

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

    def get_cat_size_map(self):
        return self.cat_size



class DataNegEnhancement(object):
    def __init__(self, f_convert=None):
        self.f_convert = f_convert if f_convert is not None else self.neg_enhancement
        
    def transform(self, df):
        neg_df = df.copy()
        neg_df = self.f_convert(neg_df)
        return pd.concat([df, neg_df])
    
    @staticmethod
    def neg_enhancement(df):
        df['Y'] = (160 / 3) - df['Y']
        df['y1'] = (160 / 3) - df['y1']
        df['y2'] = (160 / 3) - df['y2']
        for f in df.columns:
            if f.endswith('_cont_Y') or f.endswith('_cont_y1') or f.endswith('_cont_y2'):
                df[f] = -df[f]
        df['Dir'] = df['Dir'].apply(lambda x:(-(x-90)%360 + 90)%360)
        df['Orientation'] = df['Orientation'].apply(lambda x:(-(x-90)%360 + 90)%360)
        for col in ['speed_cos_Dir','acc_cos_Dir','speed_cos_Ori','acc_cos_Ori']:
            for f in df.columns:
                if col in f:
                    df[f] = -df[f]
            
        return df

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(23171, 317)
done fe 32.65510082244873


In [3]:
def train_base_model(train_df, continuous_features, category_features, label, cat_size_map, epoch=100, batch_size=1024,
                     common_embedding_size=8, specified_embedding_map=None, learning_rate=0.001, nfold=5, last_only=False):
    errs = []
    clfs = []

    oof_predictions = np.zeros((train_df.shape[0], 199))
    oof_targets = np.zeros((train_df.shape[0], 199))

    category_features = [f for f in category_features if f in cat_size_map]
    
    # fold data
    kf = KFoldByKey(nfold, shuffle=False)
    cnt = 0
    for train_index, test_index in kf.split(train_df["GameId"]):
        cnt += 1
        print(f"Training for FOLD {cnt}")
        gc.collect()
        timer = GoldenTimer()
        if (not last_only) or cnt == nfold:
            X_train, X_test = train_df.iloc[train_index].copy(), train_df.iloc[test_index].copy()
            
            data_enhancement = DataNegEnhancement()
            X_train = data_enhancement.transform(X_train)
            
            X_train[continuous_features] = sc.transform(X_train[continuous_features])
            X_test[continuous_features] = sc.transform(X_test[continuous_features])
    
            model = NLF_NN(continuous_features, category_features, label, epoch, batch_size, 
                           common_embedding_size=common_embedding_size, specified_embedding_map=specified_embedding_map,
                           learning_rate=learning_rate, 
                           player_continuous_feature=player_continuous_feature, player_category_feature=player_category_feature)
             
            model.build_model(cat_size_map)
            model.train_model(X_train, validate_df=X_test)
            
            y_pred, y_target, score = model.score(X_test)
            
            oof_predictions[test_index] = y_pred
            oof_targets[test_index] = y_target

            print(f"FOLD {cnt} score:", score)
            
            if score < 0.015 and not math.isnan(score):
                errs.append(score)
                clfs.append(model)

    oof_score = ((oof_predictions - oof_targets) ** 2).sum(axis=1).sum(axis=0) / (199 * oof_targets.shape[0])                        
    print(f"OOF Score {oof_score}, Mean Score {np.mean(errs)}, Std Score {np.std(errs)}")
    return clfs, oof_predictions, oof_targets

In [4]:
def train_base_model(train_df, continuous_features, category_features, label, cat_size_map, epoch=100, batch_size=1024,
                     common_embedding_size=8, specified_embedding_map=None, learning_rate=0.001, nfold=10, last_only=False):
    K.clear_session()
    errs = []
    clfs = []
    
    oof_predictions = np.zeros((train_df.shape[0], 199))
    oof_targets = np.zeros((train_df.shape[0], 199))

    category_features = [f for f in category_features if f in cat_size_map]
    
    # fold data
    kf = KFoldByKey(nfold, shuffle=False)
    cnt = 0
    for train_index, test_index in kf.split(train_df["GameId"]):
        cnt += 1
        print(f"Training for FOLD {cnt}")
        gc.collect()
        if (not last_only) or cnt == nfold:
            X_train, X_test = train_df.iloc[train_index].copy(), train_df.iloc[test_index].copy()
            
            data_enhancement = DataNegEnhancement()
            X_train = data_enhancement.transform(X_train)
            
            X_train[continuous_features] = sc.transform(X_train[continuous_features])
            X_test[continuous_features] = sc.transform(X_test[continuous_features])
    
            model = NLF_NN(continuous_features, category_features, label, epoch, batch_size, 
                           common_embedding_size=common_embedding_size, specified_embedding_map=specified_embedding_map,
                           learning_rate=learning_rate, 
                           player_continuous_feature=player_continuous_feature, player_category_feature=player_category_feature)
            
            model.build_model(cat_size_map)
            model.train_model(X_train, validate_df=X_test)
            
            y_pred, y_target, score = model.score(X_test)
            
            oof_predictions[test_index] = y_pred
            oof_targets[test_index] = y_target

            print(f"FOLD {cnt} score:", score)
            
            if score < 0.015 and not math.isnan(score):
                errs.append(score)
                clfs.append(model)

    oof_score = ((oof_predictions - oof_targets) ** 2).sum(axis=1).sum(axis=0) / (199 * oof_targets.shape[0])                        
    print(f"OOF Score {oof_score}, Mean Score {np.mean(errs)}, Std Score {np.std(errs)}")
    return clfs, oof_predictions, oof_targets


def is_team_feature(f):
    team_prefix = ['friend', 'ravel']
    for prefix in team_prefix:
        if f.startswith(prefix):
            return True
    return False




continuous_features = [f for f in train_df.columns if f not in exclude_feature + category_features and 
                       train_df[f].dtype != 'object' and not is_team_feature(f)]


continuous_features = [
    'X', 'Y', 'S', 'A', 'Dis', 'Distance', #'PlayerWeight',
    'Dir',
    'speed_sin_Dir', 'speed_cos_Dir', 'acc_sin_Dir', 'acc_cos_Dir',
    "x1", "y1", "x2", "y2",
    'YardLine_std_diff', #'Grass'
    "is_old", #'YardLine_std',
]

print(category_features)
print(continuous_features)

sc = preprocessing.StandardScaler()
sc.fit(train_df[continuous_features])

cfe = CategoryFeatureEncoder(category_features+friend_category_features+ravel_category_features)
train_encode_df = cfe.fit_transform(train_df)

clfs, triple_predictions, triple_targets = train_base_model(train_encode_df, continuous_features, category_features, 'Yards', 
                        cfe.get_cat_size_map(), common_embedding_size = 10, specified_embedding_map={"NflId": 256}, last_only=False)


['DefendersInTheBox', 'Position', 'OffenseFormation', 'DefensePersonnel']
['X', 'Y', 'S', 'A', 'Dis', 'Distance', 'Dir', 'speed_sin_Dir', 'speed_cos_Dir', 'acc_sin_Dir', 'acc_cos_Dir', 'x1', 'y1', 'x2', 'y2', 'YardLine_std_diff', 'is_old']
Training for FOLD 1
Train on 41736 samples, validate on 2303 samples
Epoch 1/100
41736/41736 [==============================] - 11s 275us/step - loss: 3.5841 - crps_loss_by_label_tensor: 0.0277 - val_loss: 2.8448 - val_crps_loss_by_label_tensor: 0.0138
Epoch 2/100
41736/41736 [==============================] - 6s 147us/step - loss: 2.9531 - crps_loss_by_label_tensor: 0.0140 - val_loss: 2.7388 - val_crps_loss_by_label_tensor: 0.0130
Epoch 3/100
41736/41736 [==============================] - 6s 146us/step - loss: 2.8278 - crps_loss_by_label_tensor: 0.0132 - val_loss: 2.6602 - val_crps_loss_by_label_tensor: 0.0125
Epoch 4/100
41736/41736 [==============================] - 6s 147us/step - loss: 2.7592 - crps_loss_by_label_tensor: 0.0128 - val_loss: 2.620

In [5]:
oof_score = ((triple_predictions - triple_targets) ** 2).sum(axis=1).sum(axis=0) / (199 * triple_targets.shape[0])
print('triple score', oof_score)

triple score 0.01179884270091576


In [6]:
def make_predict(test_df, models, exclude_feature=[]):
    test_df = test_df.copy()
    test_df.fillna(0, inplace=True)
    test_df = fe(test_df)
    test_df[continuous_features] = sc.transform(test_df[continuous_features])
    test_df = cfe.transform(test_df)
    
    preds_labels = np.mean([model.predict(test_df) for model in models], axis=0)
    preds_labels[0][1:] = np.max([preds_labels[0][1:], preds_labels[0][:-1]], axis=0)
    
    preds_labels[0][0] = 0
    preds_labels[0][-1] = 1
    
    return preds_labels

In [7]:
from kaggle.competitions import nflrush
env = nflrush.make_env()
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test: 
    y_pred = make_predict(test_df, clfs)
    preds_df = pd.DataFrame(data=y_pred, columns=sample_prediction_df.columns)
    env.predict(preds_df)
    
env.write_submission_file()

/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/opt/conda/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; r

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
